In [1]:
import torch
import safetensors.torch
import json
from stable_audio_tools.models.factory import create_pretransform_from_config
from stable_audio_tools import get_pretrained_model


In [2]:
from huggingface_hub import login

login()

In [3]:

device = "cuda:1" if torch.cuda.is_available() else "cpu"

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

# Save pretransform
pretransform = model.pretransform
pretransform_state_dict = model.pretransform.state_dict()

file_path = 'pretransform.safetensors'
safetensors.torch.save_file(pretransform_state_dict, file_path)

# Load the pretransform configuration
pretransform_config_path = 'model_config.json'
with open(pretransform_config_path) as f:
    pretransform_config = json.load(f)

# Create the pretransform model from the configuration
# pretransform_config['chunked'] = False
reload_pretransform = create_pretransform_from_config(pretransform_config, sample_rate=model_config["sample_rate"])
reload_pretransform = reload_pretransform.to(device)

# Check if the original pretransform and the reloaded pretransform are of the same type
print(type(pretransform) == type(reload_pretransform))  # Should print True

# Apply the state dictionary to the pretransform model
state_dict = safetensors.torch.load_file(file_path)
new_state_dict = {}
for key, value in state_dict.items():
    new_key = key.replace('model.', '')  # 'model.'
    new_state_dict[new_key] = value

reload_pretransform.load_state_dict(new_state_dict)


/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/x_transformers/x_transformers.py:435: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/x_transformers/x_transformers.py:461: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/stable_audio_tools/models/transformer.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/stable_audio_tools/models/transformer.py:151: FutureWarning: `torch.cuda.amp.autocast(args

No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:436: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:619: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/eduardo/Projects/pre_encode_audio/pre_encode_env/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


True


In [4]:

import torch
import torchaudio

# Define the path to the audio file
audio_file_path = lambda x: f'./data/babyslakh_16k/Track000{x}/mix.wav'
base_path = lambda x: f'./data/babyslakh_16k/Track000{x}/'
outputs = './data/latents/'
for x in range(1, 21):
# Load the audio file
    num = str(x)
    num = num if len(num) > 1 else f'0{num}'
    print(audio_file_path(num))
    waveform, sample_rate = torchaudio.load(audio_file_path(num), )

    waveform = waveform.to(device=device)
    preprocessed_audio = reload_pretransform.model.preprocess_audio_for_encoder(waveform, sample_rate)
    # print(preprocessed_audio.shape)
    latent = reload_pretransform.encode(preprocessed_audio)
    print(latent.shape)
    print(latent.mean())
    print(latent.var())
    with open(base_path(num) + 'latent.npy', 'wb') as outfile:
        latent.cpu().numpy().tofile(outfile)
    with open(outputs + f'Track000{num}_' + 'latent.npy', 'wb') as outfile:
        latent.cpu().numpy().tofile(outfile)


./data/babyslakh_16k/Track00001/mix.wav
torch.Size([1, 64, 5202])
tensor(-0.1301, device='cuda:1')
tensor(1.0509, device='cuda:1')
./data/babyslakh_16k/Track00002/mix.wav
torch.Size([1, 64, 4100])
tensor(-0.1283, device='cuda:1')
tensor(1.0582, device='cuda:1')
./data/babyslakh_16k/Track00003/mix.wav
torch.Size([1, 64, 4710])
tensor(-0.0863, device='cuda:1')
tensor(1.0987, device='cuda:1')
./data/babyslakh_16k/Track00004/mix.wav
torch.Size([1, 64, 5009])
tensor(-0.1086, device='cuda:1')
tensor(1.0503, device='cuda:1')
./data/babyslakh_16k/Track00005/mix.wav
torch.Size([1, 64, 5323])
tensor(-0.1147, device='cuda:1')
tensor(1.0661, device='cuda:1')
./data/babyslakh_16k/Track00006/mix.wav
torch.Size([1, 64, 5305])
tensor(-0.0934, device='cuda:1')
tensor(1.0844, device='cuda:1')
./data/babyslakh_16k/Track00007/mix.wav
torch.Size([1, 64, 5316])
tensor(-0.0822, device='cuda:1')
tensor(1.1113, device='cuda:1')
./data/babyslakh_16k/Track00008/mix.wav
torch.Size([1, 64, 5591])
tensor(-0.1165, d